In [3]:
pip install --upgrade pymupdf

Requirement already up-to-date: pymupdf in /home/lennaert/.local/lib/python3.8/site-packages (1.23.5)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import fitz

In [2]:
doc = fitz.open("../resources/testpdf/test1.pdf", filetype="pdf")

In [3]:
'''
for page in doc:
    text = page.get_text().encode("utf8")
    print(text)
'''

'\nfor page in doc:\n    text = page.get_text().encode("utf8")\n    print(text)\n'

In [4]:
print(doc.xref_object(-1))

<<
  /Size 14
  /Root 12 0 R
  /Info 13 0 R
  /ID [ <E0CFF68FAA070B8F3FAD71530AD5DB27> <E0CFF68FAA070B8F3FAD71530AD5DB27> ]
  /DocChecksum /8461BF36725D919502B9ECB1E832FBF6
>>


In [5]:
print(doc.get_xml_metadata())
print(doc.metadata)
print(doc.xref_object(doc.pdf_catalog()))


{'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Writer', 'producer': 'LibreOffice 6.4', 'creationDate': "D:20231103154149+01'00'", 'modDate': '', 'trapped': '', 'encryption': None}
<<
  /Type /Catalog
  /Pages 4 0 R
  /OpenAction [ 1 0 R /XYZ null null 0 ]
  /Lang (nl-NL)
>>


In [6]:
doc_object = {}

In [7]:
'''
Extract all objects from a pdf.
'''

xreflen = doc.xref_length()  # length of objects table
print(xreflen)
for xref in range(1, xreflen):  # skip item 0!
        print("")
        print("object %i (stream: %s)" % (xref, doc.xref_is_stream(xref)))
        print(doc.xref_object(xref, compressed=False))
        
        keys = doc.xref_get_keys(xref)
        object_keys = {}
        for key in keys:
            value = doc.xref_get_key(xref, key)
            object_keys[key] = value
        if stream := doc.xref_stream(xref):
                print(stream)
                object_keys["stream"] = stream
            
        doc_object["object%i" % (xref)] = object_keys
        
        
            

14

object 1 (stream: False)
<<
  /Type /Page
  /Parent 4 0 R
  /Resources 11 0 R
  /MediaBox [ 0 0 595.30398 841.8898 ]
  /Group <<
    /S /Transparency
    /CS /DeviceRGB
    /I true
  >>
  /Contents 2 0 R
>>

object 2 (stream: True)
<<
  /Length 3 0 R
  /Filter /FlateDecode
>>
b'0.1 w\nq 0 0.028 595.275 841.861 re\nW* n\nq 0 0 0 rg\nBT\n56.8 773.989 Td /F1 12 Tf[<01>68<02>1<03>-2<04>2<05060708>]TJ\nET\nQ\nq 0 0 0 rg\nBT\n56.8 746.389 Td /F1 12 Tf[<04>2<090A>1<0B0C050D0E0F>]TJ\nET\nQ\nQ '

object 3 (stream: False)
140

object 4 (stream: False)
<<
  /Type /Pages
  /Resources 11 0 R
  /MediaBox [ 0 0 595 841 ]
  /Kids [ 1 0 R ]
  /Count 1
>>

object 5 (stream: True)
<<
  /Length 6 0 R
  /Filter /FlateDecode
  /Length1 11264
>>
b'true\x00\x0c\x00\x80\x00\x03\x00@cmap/\x19% \x00\x00\x00\xcc\x00\x00\x01\x12cvt =O\x15;\x00\x00\x01\xe0\x00\x00\x02\xd2fpgm\x9a\xb7\\\xf4\x00\x00\x04\xb4\x00\x00\x07\xb4glyf\x96^_\x91\x00\x00\x0ch\x00\x00\x0f\x9cheadN19+\x00\x00\x1c\x04\x00\x00\x006hheaR\xfey\x

In [8]:
print(len(doc_object))
print(doc_object)
print("")
print(doc_object["object2"]['stream'].decode("utf-8"))
print("")
print(doc_object["object2"])

13
{'object1': {'Type': ('name', '/Page'), 'Parent': ('xref', '4 0 R'), 'Resources': ('xref', '11 0 R'), 'MediaBox': ('array', '[0 0 595.30398 841.8898]'), 'Group': ('dict', '<</S/Transparency/CS/DeviceRGB/I true>>'), 'Contents': ('xref', '2 0 R')}, 'object2': {'Length': ('xref', '3 0 R'), 'Filter': ('name', '/FlateDecode'), 'stream': b'0.1 w\nq 0 0.028 595.275 841.861 re\nW* n\nq 0 0 0 rg\nBT\n56.8 773.989 Td /F1 12 Tf[<01>68<02>1<03>-2<04>2<05060708>]TJ\nET\nQ\nq 0 0 0 rg\nBT\n56.8 746.389 Td /F1 12 Tf[<04>2<090A>1<0B0C050D0E0F>]TJ\nET\nQ\nQ '}, 'object3': {}, 'object4': {'Type': ('name', '/Pages'), 'Resources': ('xref', '11 0 R'), 'MediaBox': ('array', '[0 0 595 841]'), 'Kids': ('array', '[1 0 R]'), 'Count': ('int', '1')}, 'object5': {'Length': ('xref', '6 0 R'), 'Filter': ('name', '/FlateDecode'), 'Length1': ('int', '11264'), 'stream': b'true\x00\x0c\x00\x80\x00\x03\x00@cmap/\x19% \x00\x00\x00\xcc\x00\x00\x01\x12cvt =O\x15;\x00\x00\x01\xe0\x00\x00\x02\xd2fpgm\x9a\xb7\\\xf4\x00\x00\

In [9]:
def getObjectNum(xref):
    type = xref[0]
    value = xref[1]
    return int(value.split(" ")[0])

In [10]:
"""
Get root of document
"""
root = {}

root_xref = doc.pdf_catalog()
for key in doc.xref_get_keys(root_xref):
    root[key] = doc.xref_get_key(root_xref, key)

print(root)

{'Type': ('name', '/Catalog'), 'Pages': ('xref', '4 0 R'), 'OpenAction': ('array', '[1 0 R/XYZ null null 0]'), 'Lang': ('string', 'nl-NL')}


In [11]:
"""
Get root pages object
"""
pages = {}
pages_xref = getObjectNum(root['Pages'])
for key in doc.xref_get_keys(pages_xref):
    pages[key] = doc.xref_get_key(pages_xref, key)

print(pages)


{'Type': ('name', '/Pages'), 'Resources': ('xref', '11 0 R'), 'MediaBox': ('array', '[0 0 595 841]'), 'Kids': ('array', '[1 0 R]'), 'Count': ('int', '1')}


In [12]:
"""
Get resources object for root page
"""

root_resources = {}
root_resources_xref = getObjectNum(pages['Resources'])
for key in doc.xref_get_keys(root_resources_xref):
    root_resources[key] = doc.xref_get_key(root_resources_xref, key)

print(root_resources)

{'Font': ('xref', '10 0 R'), 'ProcSet': ('array', '[/PDF/Text]')}


In [13]:
"""
Get fonts
"""
import re
import binascii

fonts = {}
bfchar_pattern = r'(\d+)\s+beginbfchar\s+((?:\w+\s+\w+\s*)+)\s+endbfchar'


font_ref = {}
font_xref = getObjectNum(root_resources['Font'])
for key in doc.xref_get_keys(font_xref):
    font_ref[key] = doc.xref_get_key(font_xref, key)

print(font_ref)

# get all fonts from 'font_ref' here

for ref in font_ref:
    font_info = {}
    font_ToUniCode = {}
    font_FontDescriptor = {}
    
    
    font_info_xref = getObjectNum(font_ref[ref])
    for key in doc.xref_get_keys(font_info_xref):
        font_info[key] = doc.xref_get_key(font_info_xref, key)
    
    
    font_ToUniCode_xref = getObjectNum(font_info['ToUnicode'])
    for key in doc.xref_get_keys(font_ToUniCode_xref):
        font_ToUniCode[key] = doc.xref_get_key(font_ToUniCode_xref, key)
    if stream := doc.xref_stream(font_ToUniCode_xref):
        font_ToUniCode["Stream"] = stream.decode("utf-8") 
    
    fonts[ref] = {"Info" : font_info, "UnicodeTable": font_ToUniCode, "Description": font_FontDescriptor}
    

    temp = fonts[ref]['UnicodeTable']['Stream']
    bfchar_matches = re.findall(r'<(\w+)>\s+<(\w+)>', temp)

    cmap = {}
    for cid, value in bfchar_matches[1:]:
        cmap[f"<{cid}>"] = chr(int(value, 16))

    fonts['F1']['UnicodeTable']['Cmap'] = cmap
    print(fonts['F1'])

def FontToUnicodeInterperter():
    print("null")


{'F1': ('xref', '9 0 R')}
{'Info': {'Type': ('name', '/Font'), 'Subtype': ('name', '/TrueType'), 'BaseFont': ('name', '/BAAAAA+LiberationSerif'), 'FirstChar': ('int', '0'), 'LastChar': ('int', '15'), 'Widths': ('array', '[777 610 443 389 277 250 500 500 333 500 443 500 500 500 500 500]'), 'FontDescriptor': ('xref', '7 0 R'), 'ToUnicode': ('xref', '8 0 R')}, 'UnicodeTable': {'Length': ('int', '290'), 'Filter': ('name', '/FlateDecode'), 'Stream': '/CIDInit/ProcSet findresource begin\n12 dict begin\nbegincmap\n/CIDSystemInfo<<\n/Registry (Adobe)\n/Ordering (UCS)\n/Supplement 0\n>> def\n/CMapName/Adobe-Identity-UCS def\n/CMapType 2 def\n1 begincodespacerange\n<00> <FF>\nendcodespacerange\n15 beginbfchar\n<01> <0054>\n<02> <0065>\n<03> <0073>\n<04> <0074>\n<05> <0020>\n<06> <0070>\n<07> <0064>\n<08> <0066>\n<09> <0068>\n<0A> <0061>\n<0B> <006E>\n<0C> <006B>\n<0D> <0079>\n<0E> <006F>\n<0F> <0075>\nendbfchar\nendcmap\nCMapName currentdict /CMap defineresource pop\nend\nend\n', 'Cmap': {'<01>'

In [14]:
"""
Get next page

TODO: expand for multiple kids?
"""


kid = {}
kid_xref = int(pages['Kids'][1][1:-1].split(" ")[0])
print(kid_xref)
for key in doc.xref_get_keys(kid_xref):
    kid[key] = doc.xref_get_key(kid_xref, key)

print(kid)

1
{'Type': ('name', '/Page'), 'Parent': ('xref', '4 0 R'), 'Resources': ('xref', '11 0 R'), 'MediaBox': ('array', '[0 0 595.30398 841.8898]'), 'Group': ('dict', '<</S/Transparency/CS/DeviceRGB/I true>>'), 'Contents': ('xref', '2 0 R')}


In [15]:
"""
Get contents of page
"""
def getPageContents(page):
    contents = {}
    contents_xref = getObjectNum(kid['Contents'])
    for key in doc.xref_get_keys(contents_xref):
        contents[key] = doc.xref_get_key(contents_xref, key)     
    if stream := doc.xref_stream(contents_xref):
        contents["Stream"] = stream.decode("utf-8") 
    return contents

contents = getPageContents(kid)
print(contents)

{'Length': ('xref', '3 0 R'), 'Filter': ('name', '/FlateDecode'), 'Stream': '0.1 w\nq 0 0.028 595.275 841.861 re\nW* n\nq 0 0 0 rg\nBT\n56.8 773.989 Td /F1 12 Tf[<01>68<02>1<03>-2<04>2<05060708>]TJ\nET\nQ\nq 0 0 0 rg\nBT\n56.8 746.389 Td /F1 12 Tf[<04>2<090A>1<0B0C050D0E0F>]TJ\nET\nQ\nQ '}


In [79]:
"""
Manipulate text stream
"""

# push, pop
stack = []

"""
if token = "w":
    do 'w' things
"""

text_stream = contents["Stream"]
text_stream_parsed = text_stream.split("\n")
#print(text_stream)
#print(text_stream_parsed)
for line in text_stream_parsed:
    tokens = line.split(" ")
    print(tokens)
    for token in tokens:
        print(token)

['0.1', 'w']
0.1
w
['q', '0', '0.028', '595.275', '841.861', 're']
q
0
0.028
595.275
841.861
re
['W*', 'n']
W*
n
['q', '0', '0', '0', 'rg']
q
0
0
0
rg
['BT']
BT
['56.8', '773.989', 'Td', '/F1', '12', 'Tf[<01>68<02>1<03>-2<04>2<05060708>]TJ']
56.8
773.989
Td
/F1
12
Tf[<01>68<02>1<03>-2<04>2<05060708>]TJ
['ET']
ET
['Q']
Q
['q', '0', '0', '0', 'rg']
q
0
0
0
rg
['BT']
BT
['56.8', '746.389', 'Td', '/F1', '12', 'Tf[<04>2<090A>1<0B0C050D0E0F>]TJ']
56.8
746.389
Td
/F1
12
Tf[<04>2<090A>1<0B0C050D0E0F>]TJ
['ET']
ET
['Q']
Q
['Q', '']
Q



In [116]:
import numbers
"""
Read text stream
"""
class OperatorHandler:
    # class variable var = val
    
    def __init__(self):
        self.gState = {} #initial values here
        self.gStateStack = []
        
        self.stack = []
        self.states = [0, 1, 2]
        self.currentState = 0
      
    def handleOpForState(self, op):
        if self.currentState == 0:
            self.handleGStateOp(op)
        elif self.currentState == 1:
            self.handlePathObjectOp(op)
        elif self.currentState == 2:
            self.handleClippingPathObjectOp(op)
        
    def handleGStateOp(self, op):
        if op == 'q':
            # save current gState on stack
            pass
        elif op == 'Q':
            # restore graphics state by removing the most recently saved state from the stack and making it the current stack
            pass
        elif op == 'CM':
            # modify current transformation matrix
            pass
        elif op == 'w':
            # set linewidth in the graphics state
            value = self.stack.pop()
            if isinstance(value, numbers.Number):
                self.gState['w'] = value
                print("succes",'w', self.gState['w'])
            else:
                #error
                pass
            return
        elif op == 'J':
            # set lineCap in gState
            pass
        elif op == 'j':
            # set line join style in gState
            pass
        elif op == 'M':
            # set miter limit in gState
            pass
        elif op == 'd':
            # set line dash pattern in gState
            pass
        elif op == 'ri':
            # set the colour rendering intent in gState
            pass
        elif op == 'i':
            # set flatness tolerance in gState
            pass
        elif op == 'gs': 
            # set specified paramaters in the gState
            # for gState paramater dictionaries, used by type 2 patterns which do not have a content stream
            pass
        
        elif op == 'm':
            self.currentState = 1
            self.stack.push('m')
        elif op == 're':
            self.currentState = 1
            self.stack.push('re')
    
    def handlePathObjectOp(self, op):
        # kaas
        pass
        
    def handleClippingPathObjectOp(self, op):
        #kaas
        pass

In [117]:
handler1 = OperatorHandler()
handler1.stack = [None, 0.1, 'w']
token = handler1.stack.pop()
while token:
    handler1.handleOpForState(token)
    token = handler1.stack.pop()
    
print(handler1.stack)

succes w 0.1
[]


In [30]:
"""
extract mediabox dims.
"""
media_box = pages["MediaBox"]
cords = media_box[1][1:-1].split(" ")
print(cords)

['0', '0', '595', '841']


In [78]:
"""
Make html page based on mediabox dimensions.
"""

import codecs
f = open('test.html', 'w') 

# user space to inch to pixel (device space?)

width = (int(cords[2]) * (1/72)) / (1/96)
height = (int(cords[3]) * (1/72)) / (1/96)
print(width, height)

html_template = """ 
<html> 
<head></head> 
<body>
    <div style="width: 100%; display: flex; justify-content: center; margin-bottom: 5vh; margin-top: 5vh">
        <div>
            <div>pdf dims: {cordsW} x {cordsH}</div>
            <div>device dims: {width} x {height}<div>
            <div style="color:blue; background-color: rgb(240, 240, 240); justify-content: center; width: {width}; height: {height}">
                a
            </div>
        <div>
    <div>
</body>
</html> 
""".format(width=width, height=height, cordsW=cords[2], cordsH = cords[3])


f.write(html_template) 
  

f.close() 
  

file = codecs.open("test.html", 'r', "utf-8") 
  
# using .read method to view the html  
# code from our object 
print(file.read()) 

793.3333333333334 1121.3333333333335
 
<html> 
<head></head> 
<body>
    <div style="width: 100%; display: flex; justify-content: center; margin-bottom: 5vh; margin-top: 5vh">
        <div>
            <div>pdf dims: 595 x 841</div>
            <div>device dims: 793.3333333333334 x 1121.3333333333335<div>
            <div style="color:blue; background-color: rgb(240, 240, 240); justify-content: center; width: 793.3333333333334; height: 1121.3333333333335">
                a
            </div>
        <div>
    <div>
</body>
</html> 

